#### Libraries

In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pymavlog import MavLog
from datetime import datetime

#### Class initialization

In [18]:
class PixhawkLogger(MavLog):
    
    def __init__(self, log_path: str):
        super().__init__(log_path)
        self.message = 'None'
        self.parse()

    @staticmethod
    def timestamp_ms(TimeUS):
        timestamp = TimeUS / 1_000
        return pd.to_datetime(timestamp, unit='ms')

    @staticmethod
    def timestamp_s(TimeUS):
        timestamp = TimeUS / 1_000_000
        return pd.to_datetime(timestamp, unit='s')

    def get_message(self, message: str):
        self.message = message

    def get_field(self, field: str):
        return self.get(self.message)[field]

    def get_all_messages(self, debug: bool=False):
        all_messages = self._types
 
        if debug:
            print("-- Available Log Messages --")
            for message in all_messages:
                print(f"- {message}")

        return all_messages

    def get_all_fields(self, debug: bool = False):
        if self.message not in self._parsed_data:
            raise ValueError(f"[ERROR] No Message: {self.message}")
        
        fields = list(self._parsed_data[self.message].columns)

        if debug:
            print(f"-- Available Fields in [{self.message}] --")
            print(fields)
        
        return fields

    def get_statustext(self, debug: bool=False):
        self.message = 'MSG'
        statustext = self.get_field('Message') 

        if debug:
            for text in statustext:
                print(text)

        return statustext

    def get_gps_data(self):
        self.message = 'GPS'
        time = self.get_field('TimeUS')

        df_gps = pd.DataFrame({
            'time': self.timestamp_ms(time),
            'lat' : self.get_field('Lat'),
            'lon' : self.get_field('Lng'),
            'alt' : self.get_field('Alt')
        })

        return df_gps

    def get_att_data(self):
        self.message = 'ATT'
        time = self.get_field('TimeUS')

        df_att = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'roll'  : self.get_field('Roll'),
            'pitch' : self.get_field('Pitch'),
            'yaw'   : self.get_field('Yaw')
        })

        return df_att

    def get_ctun_data(self):
        self.message = 'CTUN'
        time = self.get_field('TimeUS')

        df_ctun = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'crt'  : self.get_field('CRt')
        })

        return df_ctun
    
    def get_imu_data(self):
        self.message = 'IMU'
        time = self.get_field('TimeUS')

        df_imu = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'gyrX'  : self.get_field('GyrX'),
            'gyrY'  : self.get_field('GyrY'),
            'gyrZ'  : self.get_field('GyrZ'),
            'accX'  : self.get_field('AccX'),
            'accY'  : self.get_field('AccY'),
            'accZ'  : self.get_field('AccZ')
        })

        return df_imu

    def get_pwm_data(self):
        self.message = 'RCOU'
        time = self.get_field('TimeUS')

        df_pwm = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'c1'  : self.get_field('C1'),
            'c2'  : self.get_field('C2'),
            'c3'  : self.get_field('C3'),
            'c4'  : self.get_field('C4')
        })

        return df_pwm

    def get_merged_data(self):
        df_gps  = self.get_gps_data().sort_values('time')
        df_att  = self.get_att_data().sort_values('time')
        df_imu  = self.get_imu_data().sort_values('time')
        df_ctun = self.get_ctun_data().sort_values('time')
        df_pwm  = self.get_pwm_data().sort_values('time')

        df_merged = pd.merge_asof(df_gps, df_att, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))
        df_merged = pd.merge_asof(df_merged, df_imu, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))
        df_merged = pd.merge_asof(df_merged, df_ctun, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))
        df_merged = pd.merge_asof(df_merged, df_pwm, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))

        return df_merged.sort_values('time')

    def get_merged_interpolate(self):
        df_gps  = self.get_gps_data()
        df_att  = self.get_att_data()
        df_imu  = self.get_imu_data()
        df_ctun = self.get_ctun_data()
        df_pwm  = self.get_pwm_data()

        df_gps  = df_gps.set_index('time')
        df_att  = df_att.set_index('time').sort_index()
        df_imu  = df_imu.set_index('time').sort_index()
        df_ctun = df_ctun.set_index('time').sort_index()
        df_pwm = df_pwm.set_index('time').sort_index()

        df_att_interp  = df_att.interpolate(method='time').reindex(df_gps.index, method='nearest')
        df_imu_interp  = df_imu.interpolate(method='time').reindex(df_gps.index, method='nearest')
        df_ctun_interp = df_ctun.interpolate(method='time').reindex(df_gps.index, method='nearest')
        df_pwm_interp = df_pwm.interpolate(method='time').reindex(df_gps.index, method='nearest')

        df_merged = pd.concat([df_gps, df_att_interp, df_imu_interp, df_ctun_interp, df_pwm_interp], axis=1)
        df_merged = df_merged.reset_index().rename(columns={'index': 'time'})

        return df_merged




#### Log circle

In [19]:
path = "log/circle_1.bin"

logs = PixhawkLogger(path)

#### All messages 

In [26]:
msg = logs.get_all_messages()

print(msg)

['UNIT', 'MULT', 'PARM', 'GPS', 'GPA', 'UBX1', 'UBX2', 'GRAW', 'GRXH', 'GRXS', 'SBPH', 'SBRH', 'SBRM', 'SBRE', 'MSG', 'RCIN', 'RCI2', 'RCOU', 'RCO2', 'RCO3', 'RSSI', 'BARO', 'BARD', 'PL', 'POWR', 'MCU', 'CMD', 'MAVC', 'RAD', 'CAM', 'TRIG', 'MNT', 'ARSP', 'BAT', 'BCL', 'MAG', 'MODE', 'RFND', 'DMS', 'BCN', 'PRX', 'PRXR', 'PM', 'SRTL', 'OABR', 'OADJ', 'SA', 'OAVG', 'SIM', 'TERR', 'ESC', 'CSRV', 'PIDR', 'PIDP', 'PIDY', 'PIDA', 'PIDS', 'PIDN', 'PIDE', 'DSTL', 'ACC', 'GYR', 'IMU', 'VIBE', 'ISBH', 'ISBD', 'RFRH', 'RFRF', 'RFRN', 'REV2', 'RSO2', 'RWA2', 'REV3', 'RSO3', 'RWA3', 'REY3', 'RISH', 'RISI', 'RASH', 'RASI', 'RBRH', 'RBRI', 'RRNH', 'RRNI', 'RGPH', 'RGPI', 'RGPJ', 'RMGH', 'RMGI', 'RBCH', 'RBCI', 'RVOH', 'ROFH', 'REPH', 'RSLL', 'REVH', 'RWOH', 'RBOH', 'XKF0', 'XKF1', 'XKF2', 'XKF3', 'XKF4', 'XKF5', 'XKFD', 'XKFM', 'XKFS', 'XKQ', 'XKT', 'XKTV', 'XKV1', 'XKV2', 'XKY0', 'XKY1', 'NKY0', 'NKY1', 'AHR2', 'AOA', 'ATT', 'ORGN', 'POS', 'RATE', 'ATSC', 'VSTB', 'MON', 'WDOG', 'RPM', 'FNCE', 'DSF', 

#### All dataframe

In [22]:
df = logs.get_merged_data()

df

,time,lat,lon,alt,roll,pitch,yaw,gyrX,gyrY,gyrZ,accX,accY,accZ,crt,c1,c2,c3,c4
0,1970-01-01 00:00:23.106147,-7.265868,112.791129,15.87,0.57,-1.57,99.48,0.000809,-0.000424,0.015505,-0.161799,-0.128529,-9.829406,NaN,1000,1000,1000,1000
1,1970-01-01 00:00:23.226824,-7.265868,112.791129,15.87,0.57,-1.57,99.50,-0.000386,-0.000353,-0.014057,-0.171069,-0.150347,-9.823412,NaN,1000,1000,1000,1000
2,1970-01-01 00:00:23.347885,-7.265868,112.791129,15.88,0.57,-1.57,99.50,0.000674,-0.001611,0.014841,-0.178722,-0.149731,-9.819493,NaN,1000,1000,1000,1000
3,1970-01-01 00:00:23.408929,-7.265867,112.791129,15.88,0.57,-1.58,99.52,0.000795,-0.000023,0.003978,-0.174291,-0.159833,-9.815162,NaN,1000,1000,1000,1000
4,1970-01-01 00:00:23.508856,-7.265867,112.791129,15.89,0.57,-1.57,99.47,-0.000088,-0.000154,0.003808,-0.171710,-0.126487,-9.830187,NaN,1000,1000,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,1970-01-01 00:02:02.628853,-7.265868,112.791100,17.77,1.03,-0.36,93.22,0.000377,0.000317,0.001816,-0.022230,-0.240980,-9.815374,NaN,1000,1000,1000,1000
828,1970-01-01 00:02:02.728946,-7.265868,112.791100,17.77,1.03,-0.36,93.22,0.000553,-0.000720,0.001275,-0.030813,-0.256754,-9.817620,NaN,1000,1000,1000,1000
829,1970-01-01 00:02:02.828849,-7.265868,112.791100,17.78,1.03,-0.37,93.22,0.000048,-0.000832,0.001458,-0.028595,-0.248790,-9.824906,NaN,1000,1000,1000,1000
830,1970-01-01 00:02:02.928849,-7.265868,112.791100,17.78,1.03,-0.36,93.23,-0.000041,-0.000845,0.001655,-0.030963,-0.244207,-9.833687,NaN,1000,1000,1000,1000


#### Log lemni

In [28]:
path = "log/lemni_1.bin"

log = PixhawkLogger(path)

#### All messages

In [29]:
log.get_all_messages()

['UNIT',
 'MULT',
 'PARM',
 'GPS',
 'GPA',
 'UBX1',
 'UBX2',
 'GRAW',
 'GRXH',
 'GRXS',
 'SBPH',
 'SBRH',
 'SBRM',
 'SBRE',
 'MSG',
 'RCIN',
 'RCI2',
 'RCOU',
 'RCO2',
 'RCO3',
 'RSSI',
 'BARO',
 'BARD',
 'PL',
 'POWR',
 'MCU',
 'CMD',
 'MAVC',
 'RAD',
 'CAM',
 'TRIG',
 'MNT',
 'ARSP',
 'BAT',
 'BCL',
 'MAG',
 'MODE',
 'RFND',
 'DMS',
 'BCN',
 'PRX',
 'PRXR',
 'PM',
 'SRTL',
 'OABR',
 'OADJ',
 'SA',
 'OAVG',
 'SIM',
 'TERR',
 'ESC',
 'CSRV',
 'PIDR',
 'PIDP',
 'PIDY',
 'PIDA',
 'PIDS',
 'PIDN',
 'PIDE',
 'DSTL',
 'ACC',
 'GYR',
 'IMU',
 'VIBE',
 'ISBH',
 'ISBD',
 'RFRH',
 'RFRF',
 'RFRN',
 'REV2',
 'RSO2',
 'RWA2',
 'REV3',
 'RSO3',
 'RWA3',
 'REY3',
 'RISH',
 'RISI',
 'RASH',
 'RASI',
 'RBRH',
 'RBRI',
 'RRNH',
 'RRNI',
 'RGPH',
 'RGPI',
 'RGPJ',
 'RMGH',
 'RMGI',
 'RBCH',
 'RBCI',
 'RVOH',
 'ROFH',
 'REPH',
 'RSLL',
 'REVH',
 'RWOH',
 'RBOH',
 'XKF0',
 'XKF1',
 'XKF2',
 'XKF3',
 'XKF4',
 'XKF5',
 'XKFD',
 'XKFM',
 'XKFS',
 'XKQ',
 'XKT',
 'XKTV',
 'XKV1',
 'XKV2',
 'XKY0',
 'XKY1',
 

#### All dataframe

In [30]:
log.get_merged_data()

,time,lat,lon,alt,roll,pitch,yaw,gyrX,gyrY,gyrZ,accX,accY,accZ,crt,c1,c2,c3,c4
0,1970-01-01 00:02:37.925412000,-7.265877,112.791078,18.00,0.65,-0.91,87.39,-0.000482,-0.001765,-0.001155,-0.063218,-0.165814,-9.798840,NaN,1000,1000,1000,1000
1,1970-01-01 00:02:38.025934000,-7.265877,112.791078,18.00,0.65,-0.91,87.39,-0.001011,-0.000583,-0.002782,-0.062686,-0.158331,-9.783065,NaN,1000,1000,1000,1000
2,1970-01-01 00:02:38.127871000,-7.265877,112.791078,18.00,0.65,-0.91,87.39,0.001036,-0.001729,-0.000435,-0.065899,-0.160682,-9.802204,NaN,1000,1000,1000,1000
3,1970-01-01 00:02:38.209129000,-7.265876,112.791078,17.99,0.66,-0.92,87.39,0.000536,-0.000369,-0.002279,-0.067912,-0.163927,-9.800107,NaN,1000,1000,1000,1000
4,1970-01-01 00:02:38.310598000,-7.265876,112.791078,17.99,0.66,-0.92,87.39,0.000150,-0.000689,-0.001945,-0.066356,-0.144809,-9.789960,NaN,1000,1000,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,1970-01-01 00:04:25.572613000,-7.265855,112.791146,17.36,0.24,-0.42,82.85,0.004651,-0.004296,-0.002677,0.021408,-0.077122,-9.806375,NaN,1000,1000,1000,1000
1075,1970-01-01 00:04:25.652385000,-7.265855,112.791146,17.37,0.25,-0.42,82.85,0.000713,-0.000447,-0.002369,0.019733,-0.092676,-9.806516,NaN,1000,1000,1000,1000
1076,1970-01-01 00:04:25.752381000,-7.265855,112.791146,17.39,0.24,-0.42,82.85,-0.000442,-0.001418,-0.002162,0.025483,-0.074516,-9.807624,NaN,1000,1000,1000,1000
1077,1970-01-01 00:04:25.832697999,-7.265855,112.791146,17.40,0.24,-0.41,82.85,-0.000030,-0.000729,0.000201,0.018730,-0.074743,-9.798866,NaN,1000,1000,1000,1000
